In [ ]:
import traci
import os
from od.env.config import (SUMO_SECONDS_PER_STEP, SUMO_SIM_GUI,
                           SUMO_SIM_STEPS, SUMO_SKIP_STEPS,
                           NET_STEPS_PER_SUMO_STEP, NET_TS_PER_NET_STEP,
                           BS_RADIUS_COLOR, BS_RADIUS)
from od.misc.interest import InterestConfig
from od.network.controller import BaseStationController
from od.network.types import BaseStationType, ResourceAllocatorType
from od.vehicle import VehicleRecorder
from od.env.config import (SUMO_SECONDS_PER_STEP, SUMO_SIM_GUI,
                           SUMO_SIM_STEPS, SUMO_SKIP_STEPS,
                           NET_STEPS_PER_SUMO_STEP, NET_TS_PER_NET_STEP,
                           BS_RADIUS_COLOR, BS_RADIUS)
from od.env.station import BS_PRESET
from od.misc.types import DebugMsgType
from od.layer import NetObjLayer
from od.misc.interest import InterestConfig
import pickle
SUMO_SIM_GUI = True

In [ ]:
def InitTraci(interest_config):
    traci.start([
        "sumo-gui" if SUMO_SIM_GUI else "sumo",
        "-c",
        os.getcwd() + "/ntust.sumocfg",
        "--quit-on-end",
        "--start",
        "--step-length",
        str(SUMO_SECONDS_PER_STEP),
        "--scale",
        str(interest_config.traffic_scale)
    ])

In [ ]:
vehicle_positions = {}
ic = InterestConfig(ResourceAllocatorType.NOMA_OPT,False,1.0)

In [ ]:
# Prepare Simulation
# - start traci
InitTraci(ic)

# - skipping until desire vehicle network condition.
for _ in range(SUMO_SKIP_STEPS):
    traci.simulationStep()

# Start Simulation
# - record vehicles
vehicle_recorders = {}
# - run
for i in range(SUMO_SIM_STEPS):
    # forward sumo simulation step
    traci.simulationStep()
    vehicles = traci.vehicle.getIDList()
    for v in vehicles:
        if(v not in vehicle_positions):
            vehicle_positions[v] = [None for _ in range(SUMO_SIM_STEPS)]
        vehicle_positions[v][i] = traci.vehicle.getPosition(v)
traci.close(wait=False)

<ipython-input-12-7e478ba0b60f>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  numpy.array([l for l in vehicle_positions.values()]).transpose()


array([[(333.7583984751095, 391.1944237412929),
        (324.22913957184073, 382.3965649932055),
        (342.318916474304, 403.3936039943514), ..., None, None, None],
       [(334.39826914677155, 391.78442891856866),
        (324.7641426291034, 382.89668990744735),
        (343.08823172827056, 404.0941869900148), ..., None, None, None],
       [(335.0496554109004, 392.3850522714786),
        (325.3141453519737, 383.4082361857099),
        (343.87061728691737, 404.8066725615857), ..., None, None, None],
       ...,
       [(416.292482275478, 466.43076563464865),
        (402.1444994792564, 453.54670686629805),
        (438.7106036333063, 491.17345315075363), ...,
        (599.0096043430004, 150.1540726897859),
        (400.5247201741285, 500.4198752312102),
        (-84.89765745853468, 1.4405055350553049)],
       [(417.1621019404127, 467.2226969672288),
        (403.01045895599475, 454.33530499721223),
        (439.72558927071844, 492.09775781616514), ...,
        (597.8065374533862, 

In [14]:
import numpy
import pandas as pd
vehicle_pos_pd = pd.DataFrame(data=numpy.array([l for l in vehicle_positions.values()]).transpose(),columns=vehicle_positions.keys())

<ipython-input-14-6f00aaf4cb9f>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  vehicle_pos_pd = pd.DataFrame(data=numpy.array([l for l in vehicle_positions.values()]).transpose(),columns=vehicle_positions.keys())


In [17]:
with open("./testPos.pickle", 'wb') as interest_statistic_report_file:
    pickle.dump(vehicle_pos_pd, interest_statistic_report_file)

In [18]:
vehicle_pos_pd

,flow_0.11,flow_0.13,flow_0.14,flow_0.15,flow_0.16,flow_0.17,flow_0.18,flow_0.19,flow_0.20,flow_0.21,...,flow_8.2,flow_8.3,flow_9.3,flow_9.4,flow_0.47,flow_0.48,flow_2.64,flow_12.1,flow_2.65,flow_0.49
0,"(333.7583984751095, 391.1944237412929)","(324.22913957184073, 382.3965649932055)","(342.318916474304, 403.3936039943514)","(315.4029288486472, 374.1457562901628)","(331.82862002061876, 393.76232122164294)","(307.3337425208601, 366.60261973159095)","(322.06324939580674, 384.73975496362755)","(313.2716646470516, 376.52131488401864)","(299.93949304401383, 359.6904192585483)","(305.1822165123885, 368.9592374347572)",...,"(280.84950369124306, 341.8449524741517)","(34.80710311201496, 111.72293958794208)","(44.822936892197774, 116.6011214758593)","(-18.632868129939496, 58.13981977534333)",None,None,None,None,None,None
1,"(334.39826914677155, 391.78442891856866)","(324.7641426291034, 382.89668990744735)","(343.08823172827056, 404.0941869900148)","(315.8321470239661, 374.5469926947185)","(332.47991440990853, 394.363385063262)","(307.6653343067586, 366.9125942471279)","(322.59941012653064, 385.24096207979824)","(313.7096896909603, 376.93078401610444)","(300.1864335853748, 359.92126114887856)","(305.52385412189756, 369.27860286237126)",...,"(280.8991529707826, 341.89136499709866)","(35.04539498808522, 111.94247478422668)","(45.17158255583671, 116.92232580573047)","(-17.786907051484803, 58.91919663167948)",None,None,None,None,None,None
2,"(335.0496554109004, 392.3850522714786)","(325.3141453519737, 383.4082361857099)","(343.87061728691737, 404.8066725615857)","(316.27442880933427, 374.9604410609946)","(333.1421184013778, 394.9745171129055)","(308.0154458347722, 367.23988115800785)","(323.15432938011304, 385.7570281599474)","(314.1575505686194, 377.34944776015396)","(300.44357293854597, 360.16163696581486)","(305.8842240700426, 369.61547942216794)",...,"(280.9555963300362, 341.94412867727397)","(35.297141140454116, 112.1744052297191)","(45.53205398543092, 117.25442511473098)","(-16.95979215361271, 59.68121065968879)",None,None,None,None,None,None
3,"(335.714839762111, 392.9983984185019)","(325.878796213989, 383.928883476946)","(344.6655840700691, 405.5306153243501)","(316.7283447888317, 375.38476516015135)","(333.81600120783145, 395.59642725846925)","(308.3730231213191, 367.57414711656907)","(323.726371671314, 386.28495215679436)","(314.61596929034454, 377.7779810568844)","(300.7130896931208, 360.4135832714269)","(306.2463305219283, 369.9539792788273)",...,"(281.02081584507187, 342.0050963744138)","(35.55981409744673, 112.41640239774267)","(45.9104423641712, 117.60303115891857)","(-16.152029822176416, 60.425395330325856)",None,None,None,None,None,None
4,"(336.39083097286647, 393.62170924020694)","(326.4561948907313, 384.46128513827745)","(345.4785602416361, 406.2709584785288)","(317.20052304569367, 375.82616097465046)","(334.5016612534873, 396.22920633328374)","(308.7490383689734, 367.9256490209674)","(324.31091892050387, 386.82441667696173)","(315.0848574224733, 378.2163012378377)","(300.9912902997751, 360.6736473078024)","(306.6253554863047, 370.3082946892366)",...,"(281.0916965631202, 342.07135620705236)","(35.83840212274329, 112.6730619105411)","(46.29848905587072, 117.96053532594344)","(-15.374016060599885, 61.142172892706654)","(-104.41925347070637, -16.544486160831223)",None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"(414.5462097479125, 464.8404980894606)","(400.40688461834213, 451.96432354655315)","(436.67165961750027, 489.3166727226803)","(386.53421352171415, 439.33098123497723)","(421.1664233619697, 475.1967071970251)","(372.93778669016854, 426.949204634005)","(402.6114952513009, 458.2995157620863)","(388.33071541692016, 445.2946111209846)","(359.6933652386275, 414.8879866644701)","(374.33149901719366, 432.54611443943173)",...,"(322.2948022172515, 380.5883317682808)","(104.59937152132572, 174.54937152132572)","(99.17405093038009, 92.47671876388749)","(22.0554987